# Libraries installation on the current python environment

# Pre-processing original data

In [37]:
import csv
def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break

In [38]:
sampleAnno = 0

## Reading from dataset

In [39]:
import os
import pandas as pd
ANNOTATIONS_PATH = 'data'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#with open(ANNOTATIONS_PATH+'/annotations.txt','r') as file:
#    annotations = file.read().splitlines()
annotations_file = ANNOTATIONS_PATH+'/annotationsVExamples.txt' #'/annotationsVSingle.txt'
with open(annotations_file,'r') as file:
    annotations = file.read().splitlines()[1:]#[line for line in file.read().splitlines() if 'no val' in line]
    print(len(annotations))

if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
    folder, trial, fragId, side, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
else:
    folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]
sampleData = os.path.join(RAW_PATH,folder,trial+'.tsv')
print(sampleData,startSec,endSec)
with open(sampleData,'r') as file:
    numMarkers, columnNames, readerBuffer = custom_header_reader(file)
    data = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))
if data['Time'].iloc[0] > float(startSec) or data['Time'].iloc[-1] < float(endSec):
    raise Exception(f'\x1b[31mNOT FULL FRAMES!!! data starts at: {data["Time"].iloc[0]} of {float(startSec)} and ends at {data["Time"].iloc[-1]} of {float(endSec)}\x1b[0m')
else:
    print("\x1b[32mSTART AND END SECONDS CHECK CORRECT!\x1b[0m")
data.head()

1
data/raw/2016-05-26/t_044.tsv 7.4 7.9
START AND END SECONDS CHECK CORRECT!


,Frame,Time,SMPTE,ARIEL X,ARIEL Y,ARIEL Z,LFHD X,LFHD Y,LFHD Z,RFHD X,...,RBHD Z,LBHD X,LBHD Y,LBHD Z,RPLM X,RPLM Y,RPLM Z,LPLM X,LPLM Y,LPLM Z
0,741,7.40,05:53:40:00,-119.270,1796.428,-726.736,-181.901,1671.816,-802.794,-55.090,...,-626.906,-198.284,1702.550,-639.897,604.744,1444.250,-612.975,-758.363,1373.064,-701.072
1,742,7.41,05:53:40:00,-118.946,1796.368,-726.426,-181.584,1671.801,-802.521,-54.729,...,-626.414,-198.187,1702.599,-639.685,604.710,1450.336,-620.183,-757.504,1379.213,-708.638
2,743,7.42,05:53:40:01,-118.624,1796.283,-726.087,-181.238,1671.738,-802.242,-54.324,...,-625.929,-198.150,1702.628,-639.439,604.854,1456.894,-626.413,-757.517,1385.792,-715.450
3,744,7.43,05:53:40:01,-118.405,1796.209,-725.720,-180.866,1671.669,-801.913,-53.960,...,-625.355,-198.127,1702.634,-639.180,604.763,1464.103,-630.937,-756.399,1393.254,-722.287
4,745,7.44,05:53:40:01,-118.233,1796.176,-725.338,-180.505,1671.587,-801.599,-53.593,...,-624.732,-198.111,1702.646,-638.897,605.747,1470.843,-637.871,-756.707,1400.724,-727.741


## Standardizing column names

In [40]:
print(len(data))
data.iloc[:,3::3].head()

51


,ARIEL X,LFHD X,RFHD X,C7 X,T5 X,T10 X,BWT X,FWT X,CLAV X,STRN X,...,RPNKY X,LTHMB X,LINDX X,LPNKY X,LFRM X,LFSH X,RBHD X,LBHD X,RPLM X,LPLM X
0,-119.270,-181.901,-55.090,-113.441,-115.078,-112.009,-117.570,-114.938,-115.469,-112.392,...,532.151,-652.537,-678.920,-714.335,-559.513,-203.859,-54.211,-198.284,604.744,-758.363
1,-118.946,-181.584,-54.729,-113.469,-115.015,-112.097,-117.749,-114.964,-115.394,-112.257,...,532.143,-651.495,-675.888,-711.713,-558.820,-203.935,-54.148,-198.187,604.710,-757.504
2,-118.624,-181.238,-54.324,-113.543,-114.898,-112.262,-117.935,-114.935,-115.352,-112.199,...,532.882,-652.942,-674.262,-710.242,-558.101,-203.983,-54.166,-198.150,604.854,-757.517
3,-118.405,-180.866,-53.960,-113.589,-114.714,-112.494,-118.154,-114.883,-115.300,-111.997,...,534.366,-655.060,-673.679,-709.764,-557.075,-204.099,-54.201,-198.127,604.763,-756.399
4,-118.233,-180.505,-53.593,-113.609,-114.512,-112.686,-118.322,-114.801,-115.266,-112.215,...,535.972,-658.809,-675.013,-710.363,-556.315,-204.144,-54.218,-198.111,605.747,-756.707


In [41]:
[colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]

['ARIEL',
 'LFHD',
 'RFHD',
 'C7',
 'T5',
 'T10',
 'BWT',
 'FWT',
 'CLAV',
 'STRN',
 'LBSH',
 'LSHO',
 'LFUPA',
 'LBUPA',
 'LIEL',
 'LELB',
 'LOWR',
 'LIWR',
 'RSHO',
 'RBSH',
 'RFUPA',
 'RBUPA',
 'RFSH',
 'RIEL',
 'RELB',
 'RFRM',
 'ROWR',
 'RIWR',
 'LFWT',
 'RFWT',
 'LBWT',
 'RBWT',
 'LFTHI',
 'LBTHI',
 'LKNE',
 'LKNI',
 'LSHN',
 'LANK',
 'LHEL',
 'LMT1',
 'LMT5',
 'RFTHI',
 'RBTHI',
 'RKNE',
 'RKNI',
 'RSHN',
 'RANK',
 'RHEL',
 'RMT1',
 'RMT5',
 'RTHMB',
 'RINDX',
 'RPNKY',
 'LTHMB',
 'LINDX',
 'LPNKY',
 'LFRM',
 'LFSH',
 'RBHD',
 'LBHD',
 'RPLM',
 'LPLM']

In [42]:
import pandas as pd
if '_A' in data.columns[3] or '_B' in data.columns[3]:
    data = data[['Frame','Time','SMPTE']+data.filter(like=f'_{side[1]}').columns.tolist()]
    data.columns = [col.replace(f'_{side[1]}','') for col in data.columns]
elif '61A' in data.columns[3] or '61B' in data.columns[3]:
    data = data[['Frame','Time','SMPTE']+data.filter(like=f'61{side[1]}').columns.tolist()]
    data.columns = [col.replace(f'61{side[1]}_','') for col in data.columns]
if '_01' in data.columns[3]:
    data.columns = [column for column in data.columns[:3]]+[column[:-5]+column[-2:] for column in data.columns[3:]]
if 'HeadFront' in data.columns[3] or 'RKneeIn' in data.columns[3]:
    mappings = pd.read_csv("data/raw/Sport42MarkersTo64.csv")
    mappings = mappings.set_index(mappings.columns[0])[mappings.columns[1]].to_dict()
    data.columns = [mappings[col[:-2]]+col[-2:] if col[:-2] in mappings.keys() else col for col in data.columns]
    

In [43]:
list(data.columns)[:3]+list(data.columns)[3::3]

['Frame',
 'Time',
 'SMPTE',
 'ARIEL X',
 'LFHD X',
 'RFHD X',
 'C7 X',
 'T5 X',
 'T10 X',
 'BWT X',
 'FWT X',
 'CLAV X',
 'STRN X',
 'LBSH X',
 'LSHO X',
 'LFUPA X',
 'LBUPA X',
 'LIEL X',
 'LELB X',
 'LOWR X',
 'LIWR X',
 'RSHO X',
 'RBSH X',
 'RFUPA X',
 'RBUPA X',
 'RFSH X',
 'RIEL X',
 'RELB X',
 'RFRM X',
 'ROWR X',
 'RIWR X',
 'LFWT X',
 'RFWT X',
 'LBWT X',
 'RBWT X',
 'LFTHI X',
 'LBTHI X',
 'LKNE X',
 'LKNI X',
 'LSHN X',
 'LANK X',
 'LHEL X',
 'LMT1 X',
 'LMT5 X',
 'RFTHI X',
 'RBTHI X',
 'RKNE X',
 'RKNI X',
 'RSHN X',
 'RANK X',
 'RHEL X',
 'RMT1 X',
 'RMT5 X',
 'RTHMB X',
 'RINDX X',
 'RPNKY X',
 'LTHMB X',
 'LINDX X',
 'LPNKY X',
 'LFRM X',
 'LFSH X',
 'RBHD X',
 'LBHD X',
 'RPLM X',
 'LPLM X']

## Working on NaNs

In [44]:
print("Total NaNs")
nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(nans) == 0 else '\x1b[31m'
print(color+str(nans)+'\x1b[0m '+ ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Total NaNs
Series([], dtype: float64) 
No Nans


In [45]:
from itertools import groupby
max_nans_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.isna().tolist())))
print("Max contiguous NaNs")
max_nans_per_column = max_nans_per_column[max_nans_per_column > 0].sort_values(ascending=False)
color = '\x1b[32m' if len(max_nans_per_column) == 0 else '\x1b[31m'
print(color+str(max_nans_per_column)+'\x1b[0m ' + ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Max contiguous NaNs
Series([], dtype: int64) 
No Nans


## Working on Zeros

In [46]:
print('Num Zeros')
zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(zeros) == 0 else '\x1b[31m'
print(color+str(zeros)+'\x1b[0m ' + ('\nNo Zeros' if len(zeros)==0 else '\nZeros found!'))
#list(zeros.index)

Num Zeros
Series([], dtype: float64) 
No Zeros


In [47]:

import numpy as np
#for name_col,num_zeros in zeros.items():
#    if num_zeros < 20:
#        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

# THIS IS A FIX FOR MISSING QTM AT DATA 58-59
for name_col,num_zeros in zeros.items():
    if num_zeros < 20:
        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)
        

In [48]:
print('Num Zeros')
zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(zeros) == 0 else '\x1b[31m'
print(color+str(zeros)+'\x1b[0m ' + ('\nNo Zeros' if len(zeros)==0 else '\nZeros found!'))
#list(zeros.index)

Num Zeros
Series([], dtype: float64) 
No Zeros


In [49]:
from itertools import groupby
max_zeros_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.eq(0.0).tolist())))
print("Max contiguous Zeros")
max_zeros_per_column = max_zeros_per_column[max_zeros_per_column > 0].sort_values(ascending=False)
color = '\x1b[32m' if len(max_zeros_per_column) == 0 else '\x1b[31m'
print(color+str(max_zeros_per_column)+'\x1b[0m ' + ('\nNo Zeros' if len(max_zeros_per_column)==0 else '\nZeros found!'))

Max contiguous Zeros
Series([], dtype: int64) 
No Zeros


## Converting to reduced marker set

![fullbodymarkers](../resources/fullmarkerset.png) ![markerstable](../resources/markersetable.png)

nono smarmella e apri tutto

In [50]:
def map_reduce_num_markers(reducedMarkerNames:list):
    markersMap = {reducedMarkerNames[0]:   ['RHEL','RMT1','RMT5','RTOE'],
                  reducedMarkerNames[1]:   ['LHEL','LMT1','LMT5','LTOE'],
                  reducedMarkerNames[2]:   ['RANK'],
                  reducedMarkerNames[3]:   ['LANK'],
                  reducedMarkerNames[4]:   ['RKNE','RKNI'],
                  reducedMarkerNames[5]:   ['LKNE','LKNI'],
                  reducedMarkerNames[6]:   ['RFWT','RBWT'],
                  reducedMarkerNames[7]:   ['RFWT','LFWT','RBWT','LBWT'],
                  reducedMarkerNames[8]:   ['LFWT','LBWT'],
                  reducedMarkerNames[9]:   ['C6','T5','T10','BWT','STRN','CLAV','FWT'],
                  reducedMarkerNames[10]:  ['RPLM','RTHMB','RINDX','RMID','RPNKY'],
                  reducedMarkerNames[11]:  ['LPLM','LTHMB','LINDX','LMID','LPNKY'],
                  reducedMarkerNames[12]:  ['ROWR','RIWR'],
                  reducedMarkerNames[13]:  ['LOWR','LIWR'],
                  reducedMarkerNames[14]:  ['RELB','RIEL','RFRM'],
                  reducedMarkerNames[15]:  ['LELB','LIEL','LFRM'],
                  reducedMarkerNames[16]:  ['RSHO'],
                  reducedMarkerNames[17]:  ['C7'],
                  reducedMarkerNames[18]:  ['LSHO'],
                  reducedMarkerNames[19]:  ['ARIEL','RFHD','LFHD','RBHD','LBHD']
                  }
    #   removing nan joints
    for nan_joint_name,_ in max_nans_per_column[::3].items():
        nan_joint_name = nan_joint_name[:-2]
        for markerFullList in markersMap.values():
            if nan_joint_name in markerFullList:
                markerFullList.remove(nan_joint_name)
                break
    # removing many contiguous zeros joints
    for zero_joint_name,contiguous_zeros in max_zeros_per_column[::3].items():
        if contiguous_zeros > 20:
            zero_joint_name = zero_joint_name[:-2]
            for markerFullList in markersMap.values():
                if zero_joint_name in markerFullList:
                    markerFullList.remove(zero_joint_name)
                    break
    originalDataColumns = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
    
    #   removing markers not present in the data markers
    for markerFullList in markersMap.values():
        for markerName in list(reversed(markerFullList)):
            if not markerName in originalDataColumns:
                markerFullList.remove(markerName)

    #   removing asymmetries across body
    for right_side_marker_indx,left_side_marker_indx in [(0,1),(2,3),(4,5),(6,8),(10,11),(12,13),(14,15)]: # indices of every marker on the left side and his symmetric
        markersLeft = markersMap[reducedMarkerNames[left_side_marker_indx]]
        markersRight = markersMap[reducedMarkerNames[right_side_marker_indx]]
        for elem in markersLeft:
            if not 'R'+elem[1:] in markersRight:
                markersLeft.remove(elem)
                if right_side_marker_indx == 6: 
                    markersMap[reducedMarkerNames[7]].remove(elem)
        for elem in markersRight:
            if not 'L'+elem[1:] in markersLeft:
                markersRight.remove(elem)
                if left_side_marker_indx == 8: 
                    markersMap[reducedMarkerNames[7]].remove(elem)

    for i,markerFullList in enumerate(markersMap.values()):
        if len(markerFullList) == 0:
            print(markersMap)
            raise Exception(f"can't reconstruct '{reducedMarkerNames[i]}' of 20 markers, because every marker of the full set is missing!!")
    return markersMap

In [51]:
reducedMarkerNames = ['left_foot',      # 0
                      'right_foot',     # 1          
                      'left_ank',       # 2      
                      'right_ank',      # 3                          
                      'left_knee',      # 4                          
                      'right_knee',     # 5                              
                      'left_hip',       # 6                          
                      'hip_central',    # 7                              
                      'right_hip',      # 8                          
                      'spine',          # 9                       
                      'left_hand',      # 10                           
                      'right_hand',     # 11                               
                      'left_wrist',     # 12                               
                      'right_wrist',    # 13                               
                      'left_elbow',     # 14                               
                      'right_elbow',    # 15                               
                      'left_shoulder',  # 16                               
                      'shoulder_center',# 17                                   
                      'right_shoulder', # 18                                   
                      'head']           # 19                       
fullMarkerNames = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
reduced20MarkersToFullMarkers = map_reduce_num_markers(reducedMarkerNames)
reduced20MarkersToFullMarkersX = {key+'_X': [elem+' X' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
reduced20MarkersToFullMarkersY = {key+'_Y': [elem+' Y' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
reduced20MarkersToFullMarkersZ = {key+'_Z': [elem+' Z' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}

posTable = data.iloc[:,3:]

posTableX = posTable.iloc[:,::3]
posTableY = posTable.iloc[:,1::3]
posTableZ = posTable.iloc[:,2::3]

reducedPosTableX = pd.concat([posTableX[reduced20MarkersToFullMarkersX[colName+'_X']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_X' for name in reducedMarkerNames])
reducedPosTableY = pd.concat([posTableY[reduced20MarkersToFullMarkersY[colName+'_Y']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Y' for name in reducedMarkerNames])
reducedPosTableZ = pd.concat([posTableZ[reduced20MarkersToFullMarkersZ[colName+'_Z']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Z' for name in reducedMarkerNames])

#reducedPosTableX = (reducedPosTableX - reducedPosTableX.min(axis=None))/(reducedPosTableX.max(axis=None)-reducedPosTableX.min(axis=None))
#reducedPosTableY = (reducedPosTableY - reducedPosTableY.min(axis=None))/(reducedPosTableY.max(axis=None)-reducedPosTableY.min(axis=None))
#reducedPosTableZ = (reducedPosTableZ - reducedPosTableZ.min(axis=None))/(reducedPosTableZ.max(axis=None)-reducedPosTableZ.min(axis=None))

def xyz_tables_to_xyz_columns(tablesList):
    xTable,yTable,zTable = tablesList
    mergedTable = pd.DataFrame()
    for joint in range(xTable.shape[1]):
        mergedTable = pd.concat([mergedTable,xTable.iloc[:,joint],yTable.iloc[:,joint],zTable.iloc[:,joint]],axis=1)
    return mergedTable

reducedPosTable = xyz_tables_to_xyz_columns([reducedPosTableX,reducedPosTableY,reducedPosTableZ])
reducedPosTable = (reducedPosTable - reducedPosTable.min(axis=None))/(reducedPosTable.max(axis=None)-reducedPosTable.min(axis=None))
reducedPosTable.head()

,left_foot_X,left_foot_Y,left_foot_Z,right_foot_X,right_foot_Y,right_foot_Z,left_ank_X,left_ank_Y,left_ank_Z,right_ank_X,...,left_shoulder_Z,shoulder_center_X,shoulder_center_Y,shoulder_center_Z,right_shoulder_X,right_shoulder_Y,right_shoulder_Z,head_X,head_Y,head_Z
0,0.358096,0.343706,0.127110,0.225119,0.343551,0.119641,0.359373,0.357824,0.162199,0.224058,...,0.133439,0.293992,0.918093,0.149125,0.237524,0.924857,0.133909,0.290775,1.000000,0.059749
1,0.358095,0.343704,0.127129,0.225110,0.343552,0.119646,0.359380,0.357813,0.162216,0.224038,...,0.133770,0.293981,0.917888,0.149378,0.237147,0.924366,0.134042,0.290865,0.999979,0.059888
2,0.358088,0.343711,0.127147,0.225091,0.343518,0.119671,0.359386,0.357794,0.162236,0.224020,...,0.134057,0.293952,0.917707,0.149594,0.236728,0.923785,0.134162,0.290950,0.999951,0.060036
3,0.358112,0.343700,0.127156,0.225088,0.343511,0.119679,0.359386,0.357814,0.162255,0.224004,...,0.134254,0.293935,0.917550,0.149749,0.236337,0.923294,0.134220,0.291023,0.999927,0.060205
4,0.358109,0.343702,0.127168,0.225088,0.343506,0.119682,0.359403,0.357794,0.162283,0.223997,...,0.134416,0.293927,0.917409,0.149867,0.235844,0.922699,0.134273,0.291093,0.999910,0.060380


## Writing to files final clean

In [52]:
import os

PROCESSED_PATH = 'data/reprocessed/'

dest_dir = os.path.join(PROCESSED_PATH,folder)
if not os.path.exists(dest_dir): 
    os.mkdir(dest_dir)
dest_file_path = os.path.join(dest_dir,trial+'_frag'+fragId+'.csv')
if not os.path.isfile(dest_file_path) or True:
    reducedPosTable.to_csv(dest_file_path,index=False)

## Plotting reduced markers

In [54]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
if sampleAnno < 45:
    posTableY = reducedPosTable.iloc[:,::3]
    posTableZ = reducedPosTable.iloc[:,1::3]
    posTableX = reducedPosTable.iloc[:,2::3]
else:
    posTableX = reducedPosTable.iloc[:,::3]
    posTableY = reducedPosTable.iloc[:,1::3]
    posTableZ = reducedPosTable.iloc[:,2::3]

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
jointsFrom = list(map(lambda x: x-1,jointsFrom))
jointsTo = list(map(lambda x: x-1,jointsTo))
edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
adjacencyMatrix = np.zeros((20,20),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True

plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ)

    # Add edges based on the weight matrix
    for i in range(20):
        for j in range(i + 1, 20):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='gray', linestyle='-', linewidth=1)

    ax.set_xlim([-0.1,0.8])#minMax[0,0],minMax[1,0]])
    ax.set_ylim([-0.1,0.8])#minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('Y', fontsize=12)
    ax.set_zlabel('Z', fontsize=12)
    ax.set_title("Movement "+str(sampleAnno))

    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()

Using matplotlib backend: TkAgg


## Plotting full markers

In [166]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
if sampleAnno < 45:
    posTableY = posTable.iloc[:,::3]
    posTableZ = posTable.iloc[:,1::3]
    posTableX = posTable.iloc[:,2::3]
else:
    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,1::3]
    posTableZ = posTable.iloc[:,2::3]
    

plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0
column_names = [ col[:-2] for col in list(posTable.columns)[::3] ]

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ)

    for i, name in enumerate(column_names):
        ax.text(filteredX[i], filteredY[i], filteredZ[i], name, fontsize=8, ha='left')


    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('Y', fontsize=12)
    ax.set_zlabel('Z', fontsize=12)
    #ax.set_title("Movement "+str(picked))

    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


In [19]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

minMax[0,:] = [np.nanmin(posTableX.iloc[:1,:].values),np.nanmin(posTableY.iloc[:1,:].values),np.nanmin(posTableZ.iloc[:1,:].values)]
minMax[1,:] = [np.nanmax(posTableX.iloc[:1,:].values),np.nanmax(posTableY.iloc[:1,:].values),np.nanmax(posTableZ.iloc[:1,:].values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=0)
    
    for i, col_name in enumerate(posTableX.columns):
        ax.text(filteredX[i], filteredY[i], filteredZ[i], col_name[:-2], horizontalalignment='center',fontsize=6, color='black')

    ax.set_xticks([minMax[0,0],minMax[1,0]])
    ax.set_yticks([minMax[0,1],minMax[1,1]])
    ax.set_zticks([minMax[0,2],minMax[1,2]])
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])
    
    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


In [242]:
plt.close('all')

# All in One

In [195]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def visualize_a_sample(posTable, sample):
    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,2::3]
    posTableZ = posTable.iloc[:,1::3]
    jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
    jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
    jointsFrom = list(map(lambda x: x-1,jointsFrom))
    jointsTo = list(map(lambda x: x-1,jointsTo))
    edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
    adjacencyMatrix = np.zeros((20,20),dtype=bool)
    adjacencyMatrix[edges[:,0],edges[:,1]] = True

    plt.close('all')
    fig = plt.figure(figsize=(10,8))
    ax:plt.Axes = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=0, azim=0)
    fig.canvas.manager.full_screen_toggle()

    
    minMax = np.zeros((2,3))
    minMax[0,:] = [posTableX.values.min(),posTableY.values.min(),posTableZ.values.min()]
    minMax[1,:] = [posTableX.values.max(),posTableY.values.max(),posTableZ.values.max()]

    # Set appropriate axis limits
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([0,1])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    # Set the window title
    fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

    # Set the initial time index
    time_index = 0
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'cyan']

    # Function to update the plot based on the slider value
    def update_plot(val):
        ax.cla()  # Clear the previous plot

        # Filter the data based on the current time index
        filteredX = posTableX.iloc[val]
        filteredY = posTableY.iloc[val]
        filteredZ = posTableZ.iloc[val]

        ax.scatter(filteredX,filteredY,filteredZ)

        # Add edges based on the weight matrix
        for i in range(20):
            for j in range(i + 1, 20):
                if adjacencyMatrix[i,j]:
                    ax.plot([filteredX[i], filteredX[j]],
                            [filteredY[i], filteredY[j]],
                            [filteredZ[i], filteredZ[j]],
                            color='gray', linestyle='-', linewidth=1)


        ax.set_xlim([minMax[0,0],minMax[1,0]])
        ax.set_ylim([0,1])
        ax.set_zlim([minMax[0,2],minMax[1,2]])

        ax.set_xlabel('X', fontsize=12)
        ax.set_ylabel('Y', fontsize=12)
        ax.set_zlabel('Z', fontsize=12)
        ax.set_title("Movement "+str(sample))

        fig.canvas.draw_idle()

    # Create a slider widget
    slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
    maxValue = posTableX.shape[0]-1
    slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


    # Define a function to update the slider value with the mouse wheel
    def on_scroll(event):
        if event.button == 'down':
            if slider.val + slider.valstep <= maxValue:
                slider.set_val(slider.val + slider.valstep)
        elif event.button == 'up':
            if slider.val - slider.valstep >= 0:
                slider.set_val(slider.val - slider.valstep)


    # Connect the mouse wheel event to the function
    fig.canvas.mpl_connect('scroll_event', on_scroll)


    # Register the update_plot function with the slider widget
    slider.on_changed(update_plot)

    # Initial plot
    update_plot(time_index)

    # Show the plot
    plt.show(block=True)

# Function to find the maximum consecutive zeros values from a mask series
def max_consecutive_zeros(series):
        consecutive_true_count = 0
        max_consecutive_true_count = 0

        for value in series:
            if value:
                consecutive_true_count += 1
                if consecutive_true_count > max_consecutive_true_count:
                    max_consecutive_true_count = consecutive_true_count
            else:
                consecutive_true_count = 0

        return max_consecutive_true_count


Using matplotlib backend: TkAgg


In [206]:
import csv
import os
import pandas as pd
from tqdm.notebook import tqdm
from itertools import groupby

def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break


ANNOTATIONS_PATH = 'data'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#with open(ANNOTATIONS_PATH+'/annotations.txt','r') as file:
#    annotations = file.read().splitlines()
with open(ANNOTATIONS_PATH+'/annotationsVSingle.txt','r') as file:
    annotations = file.read().splitlines()[1:]#[line for line in file.read().splitlines() if 'no val' in line]
    #print(len(annotations))
    
for sampleAnno in tqdm(range(len(annotations)),desc="Compressing markers and saving to csv reprocessed"):
    if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
        folder, trial, fragId, side, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
    else:
        folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]
    sampleData = os.path.join(RAW_PATH,folder,trial+'.tsv')
    with open(sampleData,'r') as file:
        numMarkers, columnNames, readerBuffer = custom_header_reader(file)
        data = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))

    if data['Time'].iloc[0] > float(startSec) or data['Time'].iloc[-1] < float(endSec):
        raise Exception(f'\x1b[31mNOT FULL FRAMES!!! data starts at: {data["Time"].iloc[0]} of {float(startSec)} and ends at {data["Time"].iloc[-1]} of {float(endSec)}\x1b[0m')
    else:
        pass


    if '_A' in data.columns[3] or '_B' in data.columns[3]:
        data = data[['Frame','Time','SMPTE']+data.filter(like=f'_{side[1]}').columns.tolist()]
        data.columns = [col.replace(f'_{side[1]}','') for col in data.columns]
    elif '61A' in data.columns[3] or '61B' in data.columns[3]:
        data = data[['Frame','Time','SMPTE']+data.filter(like=f'61{side[1]}').columns.tolist()]
        data.columns = [col.replace(f'61{side[1]}_','') for col in data.columns]
    if '_01' in data.columns[3]:
        data.columns = [column for column in data.columns[:3]]+[column[:-5]+column[-2:] for column in data.columns[3:]]
    if 'HeadFront' in data.columns[3] or 'RKneeIn' in data.columns[3]:
        mappings = pd.read_csv("data/raw/Sport42MarkersTo64.csv")
        mappings = mappings.set_index(mappings.columns[0])[mappings.columns[1]].to_dict()
        data.columns = [mappings[col[:-2]]+col[-2:] if col[:-2] in mappings.keys() else col for col in data.columns]

    nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)



    max_nans_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.isna().tolist())))

    max_nans_per_column = max_nans_per_column[max_nans_per_column > 0].sort_values(ascending=False)


    zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)

    # THIS IS A FIX FOR MISSING QTM AT DATA 58-59
    if len(zeros.index) > 15:
        for name_col,num_zeros in zeros.items():
            if num_zeros < 50:
                data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)
    if sampleAnno == 1:
        for name_col,num_zeros in zeros.items():
            if num_zeros < 10:
                data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)
    #for name_col,num_zeros in zeros.items():
    #    if num_zeros < 20:
    #        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

    zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)

    max_zeros_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.eq(0.0).tolist())))
    max_zeros_per_column = max_zeros_per_column[max_zeros_per_column > 0].sort_values(ascending=False)

    def map_reduce_num_markers(reducedMarkerNames:list):
        markersMap = {reducedMarkerNames[0]:   ['RHEL','RMT1','RMT5','RTOE'],
                      reducedMarkerNames[1]:   ['LHEL','LMT1','LMT5','LTOE'],
                      reducedMarkerNames[2]:   ['RANK'],
                      reducedMarkerNames[3]:   ['LANK'],
                      reducedMarkerNames[4]:   ['RKNE','RKNI'],#,'RSHN','RShinFrontHigh'],
                      reducedMarkerNames[5]:   ['LKNE','LKNI'],#,'LSHN','LShinFrontHigh'],
                      reducedMarkerNames[6]:   ['RFWT','RBWT'],
                      reducedMarkerNames[7]:   ['RFWT','LFWT','RBWT','LBWT'],
                      reducedMarkerNames[8]:   ['LFWT','LBWT'],
                      reducedMarkerNames[9]:   ['C6','C7','T5','T10','BWT','STRN','CLAV','FWT'],
                      reducedMarkerNames[10]:  ['RPLM','RTHMB','RINDX','RMID','RPNKY'],
                      reducedMarkerNames[11]:  ['LPLM','LTHMB','LINDX','LMID','LPNKY'],
                      reducedMarkerNames[12]:  ['ROWR','RIWR'],
                      reducedMarkerNames[13]:  ['LOWR','LIWR'],
                      reducedMarkerNames[14]:  ['RELB','RIEL','RFRM'],
                      reducedMarkerNames[15]:  ['LELB','LIEL','LFRM'],
                      reducedMarkerNames[16]:  ['RSHO'],
                      reducedMarkerNames[17]:  ['RSHO','LSHO'],#'C7'],
                      reducedMarkerNames[18]:  ['LSHO'],
                      reducedMarkerNames[19]:  ['ARIEL','RFHD','LFHD','RBHD','LBHD']
                      }
        
        # removing nan joints
        for nan_joint_name,_ in max_nans_per_column[::3].items():
            nan_joint_name = nan_joint_name[:-2]
            for markerFullList in markersMap.values():
                if nan_joint_name in markerFullList:
                    markerFullList.remove(nan_joint_name)
                    break

        # removing many contiguous zeros joints
        for zero_joint_name,contiguous_zeros in max_zeros_per_column[::3].items():
            if contiguous_zeros > 20:
                zero_joint_name = zero_joint_name[:-2]
                for markerFullList in markersMap.values():
                    if zero_joint_name in markerFullList:
                        markerFullList.remove(zero_joint_name)
                        break
        originalDataColumns = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]

        # removing markers not present in the data markers
        for markerFullList in markersMap.values():
            for markerName in list(reversed(markerFullList)):
                if not markerName in originalDataColumns:
                    markerFullList.remove(markerName)

        #   removing asymmetries across body
        for right_side_marker_indx,left_side_marker_indx in [(0,1),(2,3),(4,5),(6,8),(10,11),(12,13),(14,15)]: # indices of every marker on the left side and his symmetric
            markersLeft = markersMap[reducedMarkerNames[left_side_marker_indx]]
            markersRight = markersMap[reducedMarkerNames[right_side_marker_indx]]
            for elem in markersLeft:
                if not 'R'+elem[1:] in markersRight:
                    markersLeft.remove(elem)
                    if right_side_marker_indx == 6: 
                        markersMap[reducedMarkerNames[7]].remove(elem)
            for elem in markersRight:
                if not 'L'+elem[1:] in markersLeft:
                    markersRight.remove(elem)
                    if left_side_marker_indx == 8: 
                        markersMap[reducedMarkerNames[7]].remove(elem)

        for i,markerFullList in enumerate(markersMap.values()):
            if len(markerFullList) == 0:
                print(markersMap)
                raise Exception(f"can't reconstruct '{reducedMarkerNames[i]}' of 20 markers, because every marker of the full set is missing!!")
        return markersMap

    reducedMarkerNames = ['right_foot',      # 1
                          'left_foot',       # 2        
                          'right_ank',       # 3      
                          'left_ank',        # 4                          
                          'right_knee',      # 5                          
                          'left_knee',       # 6                              
                          'right_hip',       # 7                          
                          'hip_central',     # 8                              
                          'left_hip',        # 9                          
                          'spine',           # 10                       
                          'right_hand',      # 11                           
                          'left_hand',       # 12                               
                          'right_wrist',     # 13                               
                          'left_wrist',      # 14                               
                          'right_elbow',     # 15                               
                          'left_elbow',      # 16                               
                          'right_shoulder',  # 17                               
                          'shoulder_center', # 18                                   
                          'left_shoulder',   # 19                                   
                          'head']            # 20                       
    fullMarkerNames = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
    reduced20MarkersToFullMarkers = map_reduce_num_markers(reducedMarkerNames)
    reduced20MarkersToFullMarkersX = {key+'_X': [elem+' X' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
    reduced20MarkersToFullMarkersY = {key+'_Y': [elem+' Y' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
    reduced20MarkersToFullMarkersZ = {key+'_Z': [elem+' Z' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}

    posTable = data.iloc[:,3:]

    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,1::3]
    posTableZ = posTable.iloc[:,2::3]

    reducedPosTableX = pd.concat([posTableX[reduced20MarkersToFullMarkersX[colName+'_X']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_X' for name in reducedMarkerNames])
    reducedPosTableY = pd.concat([posTableY[reduced20MarkersToFullMarkersY[colName+'_Y']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Y' for name in reducedMarkerNames])
    reducedPosTableZ = pd.concat([posTableZ[reduced20MarkersToFullMarkersZ[colName+'_Z']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Z' for name in reducedMarkerNames])

    def xyz_tables_to_xyz_columns(tablesList):
        xTable,yTable,zTable = tablesList
        mergedTable = pd.DataFrame()
        for joint in range(xTable.shape[1]):
            mergedTable = pd.concat([mergedTable,xTable.iloc[:,joint],yTable.iloc[:,joint],zTable.iloc[:,joint]],axis=1)
        return mergedTable

    reducedPosTable = xyz_tables_to_xyz_columns([reducedPosTableX,reducedPosTableY,reducedPosTableZ])
    reducedPosTable = (reducedPosTable - reducedPosTable.min(axis=None))/(reducedPosTable.max(axis=None)-reducedPosTable.min(axis=None))
    #reducedPosTable = reducedPosTable.rolling(window=2).mean().fillna(method='bfill')
    
    if reducedPosTable.eq(0.0).apply(max_consecutive_zeros).max() > 1:
        print(sampleAnno, reducedPosTable.eq(0.0).apply(max_consecutive_zeros).max())
        visualize_a_sample(reducedPosTable,sampleAnno)

    PROCESSED_PATH = 'data/reprocessed/'

    dest_dir = os.path.join(PROCESSED_PATH,folder)
    if not os.path.exists(dest_dir): 
        os.mkdir(dest_dir)
    dest_file_path = os.path.join(dest_dir,trial+'_frag'+fragId+'.csv')
    if not os.path.isfile(dest_file_path) or True:
        reducedPosTable.to_csv(dest_file_path,index=False)

Compressing markers and saving to csv reprocessed:   0%|          | 0/60 [00:00<?, ?it/s]